In [126]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
df = pd.read_csv(r"/Users/nhmai/Downloads/archive/wh_staff_dataset.csv")
df.head()

,year,name,gender,status,salary,pay_basis,position_title
0,1997,"Abedin,Huma M.",Female,Employee,27500.0,Per Annum,OFFICE MANAGER
1,1997,"Abrams,Lori E.",Female,Employee,35000.0,Per Annum,"DEPUTY DIRECTOR, WHITE HOUSE GIFTS"
2,1997,"Alcorn,Brian A.",Male,Employee,25000.0,Per Annum,ASSISTANT DIRECTOR FOR ADVANCE
3,1997,"Allen,Jeannetta Pam",Female,Employee,22000.0,Per Annum,STAFF ASSISTANT
4,1997,"Allison,Donna Tate",Female,Employee,27624.0,Per Annum,WHITE HOUSE TELEPHONE OPERATOR


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10600 entries, 0 to 10599
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   year            10600 non-null  int64  
 1   name            10597 non-null  object 
 2   gender          10597 non-null  object 
 3   status          10600 non-null  object 
 4   salary          10595 non-null  float64
 5   pay_basis       10593 non-null  object 
 6   position_title  10598 non-null  object 
dtypes: float64(1), int64(1), object(5)
memory usage: 579.8+ KB


In [6]:
df.describe()

,year,salary
count,10600.000000,10595.000000
mean,2008.426132,75185.340632
std,6.813933,40155.241737
min,1997.000000,0.000000
25%,2002.000000,43600.000000
50%,2009.000000,61500.000000
75%,2014.000000,101957.000000
max,2020.000000,239595.000000


In [8]:
df.isnull().sum()

year              0
name              3
gender            3
status            0
salary            5
pay_basis         7
position_title    2
dtype: int64

In [9]:
#No name/gender/salary - employee is also only detailee so we can drop the data
df = df.dropna(subset = ['name', 'salary', 'pay_basis', 'position_title'], axis = 0)
df.isnull().sum()

year              0
name              0
gender            0
status            0
salary            0
pay_basis         0
position_title    0
dtype: int64

In [10]:
#Creating new column with president era
#Clinton (1997-2000), Bush (2001-2008), Obama (2009-2016), Trump (2017-2020)

def president_range(year):
    if 1997 <= year <= 2000:
        return 'Clinton Era'
    elif 2001 <= year <= 2008:
        return 'Bush Era'
    elif 2009 <= year <= 2016:
        return 'Obama Era'
    else:
        return 'Trump Era'

df['president_era'] = df['year'].apply(lambda year: president_range(year))
df

,year,name,gender,status,salary,pay_basis,position_title,president_era
0,1997,"Abedin,Huma M.",Female,Employee,27500.0,Per Annum,OFFICE MANAGER,Clinton Era
1,1997,"Abrams,Lori E.",Female,Employee,35000.0,Per Annum,"DEPUTY DIRECTOR, WHITE HOUSE GIFTS",Clinton Era
2,1997,"Alcorn,Brian A.",Male,Employee,25000.0,Per Annum,ASSISTANT DIRECTOR FOR ADVANCE,Clinton Era
3,1997,"Allen,Jeannetta Pam",Female,Employee,22000.0,Per Annum,STAFF ASSISTANT,Clinton Era
4,1997,"Allison,Donna Tate",Female,Employee,27624.0,Per Annum,WHITE HOUSE TELEPHONE OPERATOR,Clinton Era
...,...,...,...,...,...,...,...,...
10595,2020,"Young,Stewart B.",Male,Employee,120000.0,Per Annum,SPECIAL ASSISTANT TO THE PRESIDENT AND DIRECTO...,Trump Era
10596,2020,"Zadrozny,John A.",Male,Employee,158000.0,Per Annum,DEPUTY ASSISTANT TO THE PRESIDENT AND ADVISOR ...,Trump Era
10597,2020,"Zakaria,Hannah",Female,Employee,53000.0,Per Annum,ASSOCIATE LOGISTICS AND OPERATIONS MANAGER,Trump Era
10598,2020,"Ziegler,Garrett M.",Male,Employee,53000.0,Per Annum,SENIOR POLICY ANALYST,Trump Era


In [100]:
#Insight: 1741 employees total during Clinton Era: 267 unqiue female employees, 181 unique male employees
#838 duplicate names that have worked multiple years under Clinton

df.loc[df['president_era'] == 'Clinton Era', 'name'].count() #-- 1741
df.loc[df['president_era'] == 'Clinton Era', 'name'].nunique() # -- 903

clinton_era = df.loc[df['president_era'] == 'Clinton Era']

clinton_duplicate_names = clinton_era.loc[clinton_era['name'].duplicated() == True] # -- 838 names that are duplicated
clinton_unique_employees = clinton_era.drop_duplicates('name')

clinton_unique_employees['gender'].value_counts() # -- 538: female; 365: male (ratio: 1.47:1 for female to male)

Female    538
Male      365
Name: gender, dtype: int64

In [101]:
#Insight: 1055 total female employees (considers female employees that have worked multiple years under Clinton)
#Average salary is roughly $53,540, Min salary is $22,000 (disregarding the $1 salary outlier)
#Max salary is $126,825; Low paying position titles are Staff Assistant, Admin Support Staff
#High paying position titles are Assistant to President & Deputy of Chief of Staff
#Hypothesis - during this era, lots of female jobs are "assistants"/secretarial jobs

clinton_era.loc[clinton_era['gender'] == 'Female'].describe()

,year,salary
count,1055.000000,1055.000000
mean,1998.531754,53539.685308
std,1.120232,27103.618893
min,1997.000000,1.000000
25%,1998.000000,32500.000000
50%,1999.000000,45000.000000
75%,2000.000000,68570.000000
max,2000.000000,126825.000000


In [102]:
clinton_era.loc[clinton_era['gender'] == 'Female', ('name', 'salary', 'position_title')].sort_values(by = 'salary').tail(10)

,name,salary,position_title
292,"Radd,Victoria L.",125000.0,ASSISTANT TO THE PRESIDENT & CHIEF OF STAFF TO...
1066,"Lewis,Ann F.",125000.0,COUNSELOR TO THE PRESIDENT
1363,"Cahill,Mary E.",125000.0,ASSISTANT TO THE PRESIDENT AND DIRECTOR OF PUB...
158,"Higgins,Kathryn O'Leary",125000.0,ASSISTANT TO THE PRESIDENT AND CABINET SECRETARY
1697,"Ucelli,Loretta M.",125000.0,ASSISTANT TO THE PRESIDENT AND DIRECTOR OF COM...
1684,"Streett,Stephanie S.",125000.0,ASSISTANT TO THE PRESIDENT & DIRECTOR OF PRESI...
1411,"Echaveste,Maria",125000.0,ASSISTANT TO THE PRESIDENT AND DEPUTY CHIEF OF...
652,"Mathews,Sylvia M.",125000.0,ASSISTANT TO THE PRESIDENT & DEPUTY CHIEF OF S...
961,"Echaveste,Maria",125000.0,ASSISTANT TO THE PRESIDENT AND DEPUTY CHIEF OF...
1687,"Tates,Macias Cheryl",126825.0,SPECIAL ASSISTANT TO THE PRESIDENT & ASSOCIATE...


In [103]:
#Insight: 686 Male emaployees, Min salary is $21,500 (disregrding $0/$1 outliers), Max salary is $130,000
#Avg salary is roughly $63,805
#Positions and salaries are similar to the female counterpart; same position titles get paid the same despite gender

clinton_era.loc[clinton_era['gender'] == 'Male'].describe()

,year,salary
count,686.000000,686.000000
mean,1998.507289,63805.386297
std,1.117520,33802.930818
min,1997.000000,0.000000
25%,1998.000000,32630.250000
50%,1999.000000,53018.000000
75%,1999.750000,92500.000000
max,2000.000000,130000.000000


In [104]:
clinton_era.loc[clinton_era['gender'] == 'Male', ('name', 'salary', 'position_title')].sort_values(by = 'salary').tail(10)

,name,salary,position_title
774,"Sosnik,Douglas B.",125000.0,COUNSELOR TO THE PRESIDENT
32,"Berger,Samuel R.",125000.0,ASSISTANT TO THE PRESIDENT FOR NATIONAL SECURI...
784,"Stein,Lawrence Joel",125000.0,ASSISTANT TO THE PRESIDENT & DIRECTOR OF LEGIS...
787,"Stern,Todd",125000.0,ASSISTANT TO THE PRESIDENT FOR SPECIAL PROJECTS
1546,"Marshall,Thurgood,Jr.",125000.0,ASSISTANT TO THE PRESIDENT AND CABINET SECRETARY
107,"Emanuel,Rahm I.",125000.0,SENIOR ADVISOR TO THE PRESIDENT FOR POLICY & S...
1542,"Mackay,Kenneth H.,Jr.",125000.0,ASSISTANT TO THE PRESIDENT AND SPECIAL ENVOY T...
173,"Ibarra,Mickey",125000.0,ASSISTANT TO THE PRESIDENT AND DIRECTOR OF INT...
1601,"Podesta,John D.",130000.0,CHIEF OF STAFF TO THE PRESIDENT
1144,"Podesta,John D.",130000.0,CHIEF OF STAFF TO THE PRESIDENT


In [105]:
#Insight: 3535 total employees during Bush Era; 475 unique females, 415 unique male employees
#1914 employees that have worked multiple years with Bush, 890 unqiue employees total

df.loc[df['president_era'] == 'Bush Era', 'name'].count() # -- 3535
df.loc[df['president_era'] == 'Bush Era', 'name'].nunique() # -- 1621

bush_era = df.loc[df['president_era'] == 'Bush Era']

bush_duplicate_names = bush_era.loc[bush_era['name'].duplicated() == True] # -- 1914 names that are duplicated
bush_unique_employees = bush_era.drop_duplicates('name')

bush_unique_employees['gender'].value_counts() # -- 845: female; 776: male (ratio: 1.09:1 female to male)

Female    845
Male      776
Name: gender, dtype: int64

In [38]:
#Insight: 1870 female employees total (duplicates are counted), Min salary: $23,621 (disregard $582 outlier)
#Max salary: $172,200, Avg salary: roughly $58,765

bush_era.loc[bush_era['gender'] == 'Female'].describe()

,year,salary
count,1870.000000,1870.000000
mean,2004.416578,58764.695722
std,2.259808,30289.303207
min,2001.000000,582.000000
25%,2002.000000,37000.000000
50%,2004.000000,49231.000000
75%,2006.000000,68559.000000
max,2008.000000,172200.000000


In [57]:
#Lowest paying titles of female employees during Bush Era (Correspondence Analyst, Staff Analyst, Reception, etc.)
#Highest paying titles of female employees during Bush Era (Assistant to President ...)
bush_era.loc[bush_era['gender'] == 'Female', ('name', 'salary', 'position_title')].sort_values(by = 'salary').head(10)
bush_era.loc[bush_era['gender'] == 'Female', ('name', 'salary', 'position_title')].sort_values(by = 'salary').tail(10)

,name,salary,position_title
4354,"Wallace,Nicolle",165200.0,ASSISTANT TO THE PRESIDENT FOR COMMUNICATIONS
4379,"Wolff,Candida Perotti",165200.0,ASSISTANT TO THE PRESIDENT FOR LEGISLATIVE AFF...
4273,"Renner,Liza Wright",165200.0,ASSISTANT TO THE PRESIDENT FOR PRESIDENTIAL PE...
4642,"McBride,Anita B.",168000.0,ASSISTANT TO THE PRESIDENT AND CHIEF OF STAFF ...
4820,"Wolff,Candida Perotti",168000.0,ASSISTANT TO THE PRESIDENT FOR LEGISLATIVE AFF...
4785,"Townsend,Frances Fragos",168000.0,ASSISTANT TO THE PRESIDENT FOR HOMELAND SECURI...
4721,"Renner,Liza Wright",168000.0,ASSISTANT TO THE PRESIDENT FOR PRESIDENTIAL PE...
5145,"Perino,Dana M.",172200.0,ASSISTANT TO THE PRESIDENT AND PRESS SECRETARY
4979,"Gregor,Joie A.",172200.0,ASSISTANT TO THE PRESIDENT FOR PRESIDENTIAL PE...
5087,"McBride,Anita B.",172200.0,ASSISTANT TO THE PRESIDENT AND CHIEF OF STAFF ...


In [58]:
#Insight: 1665 total male employees, Average salary: $75,401; Min salary: $25,000 (taking out outliers)
#Max salary: $183,372
bush_era.loc[bush_era['gender'] == 'Male'].describe()

,year,salary
count,1665.000000,1665.000000
mean,2004.555556,75401.133333
std,2.323468,41037.876254
min,2001.000000,0.000000
25%,2002.000000,40000.000000
50%,2005.000000,60000.000000
75%,2007.000000,108000.000000
max,2008.000000,183372.000000


In [61]:
#Lowest paying titles of male employees during Bush Era (Correspondence Analyst, Staff Assistant)
#Highest paying titles of male employees during Bush Era (Assistant to President..., Senior Director Protection)
bush_era.loc[bush_era['gender'] == 'Male', ('name', 'salary', 'position_title')].sort_values(by = 'salary').head(10)
bush_era.loc[bush_era['gender'] == 'Male', ('name', 'salary', 'position_title')].sort_values(by = 'salary').tail(10)

,name,salary,position_title
4869,"Bolten,Joshua B.",172200.0,ASSISTANT TO THE PRESIDENT AND CHIEF OF STAFF
5017,"Jackson,Barry S.",172200.0,ASSISTANT TO THE PRESIDENT FOR STRATEGIC INITI...
5244,"Wainstein,Kenneth L.",172200.0,ASSISTANT TO THE PRESIDENT FOR HOMELAND SECURI...
5101,"Meyer,Daniel P.",172200.0,ASSISTANT TO THE PRESIDENT FOR LEGISLATIVE AFF...
5223,"Thiessen,Marc A.",172200.0,ASSISTANT TO THE PRESIDENT FOR SPEECHWRITING
5215,"Sullivan,Kevin F.",172200.0,ASSISTANT TO THE PRESIDENT FOR COMMUNICATIONS
4999,"Hennessey,Keith B.",172200.0,ASSISTANT TO THE PRESIDENT FOR ECONOMIC POLICY...
5278,"Zinsmeister,Walter Karl",172200.0,ASSISTANT TO THE PRESIDENT FOR DOMESTIC POLICY
2689,"Bernard,Kenneth",174000.0,SPECIAL ASSISTANT TO THE PRESIDENT AND SENIOR ...
2426,"Lawlor,Bruce M.",183372.0,SENIOR DIRECTOR PROTECTION AND PREVENTION


In [106]:
df.loc[df['president_era'] == 'Obama Era', 'name'].count() # -- 3740 employees
df.loc[df['president_era'] == 'Obama Era', 'name'].nunique() # -- 1525 unqiue employees

obama_era = df.loc[df['president_era'] == 'Obama Era']

obama_duplicate_names = obama_era.loc[obama_era['name'].duplicated() == True] # -- 2215 names that are duplicated
obama_unique_employees = obama_era.drop_duplicates('name')

obama_unique_employees['gender'].value_counts() #Female: 826, Male: 699 (1:18 female to male ratio)

Female    826
Male      699
Name: gender, dtype: int64

In [15]:
obama_era.loc[obama_era['gender'] == 'Female'].describe()

,year,salary
count,1928.000000,1928.000000
mean,2012.605809,78840.604253
std,2.333219,38231.821476
min,2009.000000,0.000000
25%,2011.000000,47443.250000
50%,2013.000000,65000.000000
75%,2015.000000,102000.000000
max,2016.000000,176461.000000


In [16]:
#Min salary (female): $21,000/36,000 (Staff Assistant, Correspondence Analayst) (outlier $0)
#Max salary (female): $176,461 (Assistant to the President...)
obama_era.loc[obama_era['gender'] == 'Female', ('name', 'salary', 'position_title')].sort_values(by = 'salary').head(20)
obama_era.loc[obama_era['gender'] == 'Female', ('name', 'salary', 'position_title')].sort_values(by = 'salary').tail(10)

,name,salary,position_title
8718,"Haines,Avril D.",174714.0,ASSISTANT TO THE PRESIDENT AND DEPUTY NATIONAL...
8854,"Murray,Shailagh J.",174714.0,ASSISTANT TO THE PRESIDENT AND SENIOR ADVISOR
8889,"Psaki,Jennifer R.",174714.0,ASSISTANT TO THE PRESIDENT AND DIRECTOR OF COM...
8615,"Canegallo,Kristie A.",176461.0,ASSISTANT TO THE PRESIDENT AND DEPUTY CHIEF OF...
8903,"Rice,Susan E.",176461.0,ASSISTANT TO THE PRESIDENT AND NATIONAL SECURI...
8745,"Jarrett,Valerie B.",176461.0,SENIOR ADVISOR AND ASSISTANT TO THE PRESIDENT ...
8842,"Monaco,Lisa O.",176461.0,ASSISTANT TO THE PRESIDENT FOR HOMELAND SECURI...
8851,"Muñoz,Cecilia",176461.0,ASSISTANT TO THE PRESIDENT AND DIRECTOR OF THE...
8966,"Tchen,Christina M.",176461.0,ASSISTANT TO THE PRESIDENT AND CHIEF OF STAFF ...
8599,"Breckenridge,Anita J.",176461.0,ASSISTANT TO THE PRESIDENT AND DEPUTY CHIEF OF...


In [27]:
#Min salary for male employees: $36,000 (Volunteer Coord, Correspondence Analyst, etc.) (Outlier of $0)
#Max salary for male employees: $225,000 (Senior Policy Advisor)
#Things to note: female and male got paid the same for similar roles (Assistant to President...)
obama_era.loc[obama_era['gender'] == 'Male', ('name', 'salary', 'position_title')].sort_values(by = 'salary').head(20)
obama_era.loc[obama_era['gender'] == 'Male', ('name', 'salary', 'position_title')].sort_values(by = 'salary').tail(10)

,name,salary,position_title
8902,"Rhodes,Benjamin J.",176461.0,ASSISTANT TO THE PRESIDENT AND DEPUTY NATIONAL...
8747,"Johnson,Broderick D.",176461.0,ASSISTANT TO THE PRESIDENT AND CABINET SECRETARY
8761,"Keenan,Cody S.",176461.0,ASSISTANT TO THE PRESIDENT AND DIRECTOR OF SPE...
8822,"McDonough,Denis R.",176461.0,ASSISTANT TO THE PRESIDENT AND CHIEF OF STAFF
8640,"Cushman,Chase M.",176461.0,ASSISTANT TO THE PRESIDENT AND DIRECTOR OF SCH...
9018,"Zients,Jeffrey D.",176461.0,ASSISTANT TO THE PRESIDENT FOR ECONOMIC POLICY...
5932,"Hash,Michael M.",179700.0,DEPUTY DIRECTOR
6024,"Love,Timothy P.",179700.0,POLICY DIRECTOR
5570,"Marcozzi,David E.",192934.0,"DIRECTOR, PUBLIC HEALTH POLICY"
7598,"Wheeler,Seth F.",225000.0,SENIOR POLICY ADVISOR


In [107]:
df.loc[df['president_era'] == 'Trump Era', 'name'].count() # -- 1576
df.loc[df['president_era'] == 'Trump Era', 'name'].nunique() # -- 824

trump_era = df.loc[df['president_era'] == 'Trump Era']

trump_duplicate_names = trump_era.loc[trump_era['name'].duplicated() == True] # -- 752 names that are duplicated
trump_unique_employees = trump_era.drop_duplicates('name')

trump_unique_employees['gender'].value_counts() # -- 440: male; 384: female (ratio: 0.87:1 female to male)
#Thing to note: first era where there were more male employees than female

Male      440
Female    384
Name: gender, dtype: int64

In [43]:
trump_era.loc[trump_era['gender'] == 'Female'].describe()

,year,salary
count,741.000000,741.000000
mean,2018.538462,86955.126856
std,1.106586,40220.187185
min,2017.000000,0.000000
25%,2018.000000,53000.000000
50%,2019.000000,74900.000000
75%,2020.000000,115000.000000
max,2020.000000,183000.000000


In [50]:
#Min salary for female employees: $40,000 (outlier of $0) (More Associate Directors, no more Correspondence Analysts)
#Max salary for female employees: $183,000 (Assistant to President...)
trump_era.loc[trump_era['gender'] == 'Female', ('name', 'salary', 'position_title')].sort_values(by = 'salary').head(10)
trump_era.loc[trump_era['gender'] == 'Female', ('name', 'salary', 'position_title')].sort_values(by = 'salary').tail(10)

,name,salary,position_title
10102,"Rollins,Brooke L.",183000.0,ASSISTANT TO THE PRESIDENT FOR STRATEGIC INITI...
10113,"Sanders,Sarah H.",183000.0,ASSISTANT TO THE PRESIDENT AND PRESS SECRETARY
10120,"Schlapp,Mercedes Viana",183000.0,ASSISTANT TO THE PRESIDENT AND SENIOR ADVISOR ...
10348,"Hicks,Hope C.",183000.0,ASSISTANT TO THE PRESIDENT AND COUNSELOR TO TH...
10328,"Grisham,Stephanie A.",183000.0,ASSISTANT TO THE PRESIDENT AND CHIEF OF STAFF ...
10317,"Gilmartin,Kayleigh M.",183000.0,ASSISTANT TO THE PRESIDENT AND PRESS SECRETARY
10248,"Conway,Kellyanne E.",183000.0,ASSISTANT TO THE PRESIDENT AND SENIOR COUNSELOR
10293,"Farah,Alyssa A.",183000.0,ASSISTANT TO THE PRESIDENT AND DIRECTOR OF STR...
10552,"Swonger,Amy H.",183000.0,ASSISTANT TO THE PRESIDENT AND ACTING DIRECTOR...
9913,"Grisham,Stephanie A.",183000.0,ASSISTANT TO THE PRESIDENT AND DEPUTY CHIEF OF...


In [108]:
#Min salary for male employees: $30,000 (outlier of $0) (Assistant to President... [might be incorrect data])
#Max salary for male employees: $239,595 (Senior Adviser to the Chief of Staff)
trump_era.loc[trump_era['gender'] == 'Male', ('name', 'salary', 'position_title')].sort_values(by = 'salary').head(20)
trump_era.loc[trump_era['gender'] == 'Male', ('name', 'salary', 'position_title')].sort_values(by = 'salary').tail(10)

,name,salary,position_title
10456,"Navarro,Peter K.",183000.0,ASSISTANT TO THE PRESIDENT FOR TRADE AND MANUF...
10462,"O'Brien,Robert C.",183000.0,ASSISTANT TO THE PRESIDENT AND NATIONAL SECURI...
9914,"Grogan,Joseph J.",183000.0,ASSISTANT TO THE PRESIDENT AND DIRECTOR OF THE...
9910,"Greenblatt,Jason D.",183000.0,ASSISTANT TO THE PRESIDENT AND SPECIAL REPRESE...
10489,"Pottinger,Matthew F.",183000.0,ASSISTANT TO THE PRESIDENT AND DEPUTY NATIONAL...
9873,"Eisenberg,John A.",183000.0,DEPUTY COUNSEL TO THE PRESIDENT FOR NATIONAL S...
9168,"House,Mark S.",187100.0,SENIOR POLICY ADVISOR
10224,"Brown,Peter J.",190289.0,SPECIAL REPRESENTATIVE FOR DISASTER RECOVERY
10048,"Mulvaney,John M.",203500.0,ACTING CHIEF OF STAFF
9849,"Czwartacki,John S.",239595.0,SENIOR ADVISOR TO THE CHIEF OF STAFF FOR STRAT...
